In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('../data/Train_dataset.csv')
train.head(1)

,Registration Number,Annual Turnover,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [4]:
from kp_auto_ml import pre_processing as pp

In [5]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [6]:
train.head(1)

,registration_number,annual_turnover,cuisine,city,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,...,overall_restaurant_rating,live_music_rating,comedy_gigs_rating,value_deals_rating,live_sports_rating,ambience,lively,service,comfortablility,privacy
0,60001,42000000,"indian,irish",Bangalore,Near Business Hub,14/02/09,84.3,Not Specific,95.8,1,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6


In [7]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='city',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.city.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='endorsed_by',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.endorsed_by.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_theme',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_theme.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='restaurant_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cuisine',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
    )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=['opening_day_of_restaurant','registration_number'],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='annual_turnover'
)

In [8]:
pre_processed_df = pp.process(train,model_config)

Dropping columns whose values are nullable greater than 0.7
Removing unusable values like " " and "?"
Filling NA values with median and mode for numeria and non-numeric columns.
Performing IQR based outliers cleaning on target. annual_turnover
Changing datatype of numeric columns.
Performing label encoding on columns provided.
Encoding using dummies.


In [31]:
from kp_auto_ml import model_training_data_prep as dp
from kp_auto_ml import model_training_helper as mth

In [34]:
# Assuming dp.ModelTrainingData(pre_processed_df, dp.ScalerType.STANDARD_SCALER, .95) returns an instance of YourClass

# Create an instance of YourClass
your_instance = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.STANDARD_SCALER, .90,use_pca=True,use_polynomials=True)

# Generate permutations and assign the result to a single variable
model, param = mth.get_model_and_param(power=mth.ModelPower.HIGH,model_and_param=mth.ModelAndParam.Ridge_Regression)
for permuted_data in your_instance.generate_permutations_train(min_columns=len(pre_processed_df.columns)-1):
    X_train_permuted, X_val_permuted, X_test_permuted = permuted_data
    best_param,best_model,score = mth.train_test_random_search_regression(model=model,param_distributions=param,X_train=X_train_permuted,y_train=your_instance.Y_train,X_test=X_val_permuted,y_test=your_instance.Y_test)
    # print(X_train_permuted[0][len(X_train_permuted[0])-1])
    print(score)

Ridge params: {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
Total Permutations: 1


InvalidParameterError: The 'estimator' parameter of RandomizedSearchCV must be an object implementing 'fit'. Got {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']} instead.